In [1]:
import tensorflow as tf
import canton as ct
import numpy as np

testing GRU implementation with tf.scan.

In [7]:
a = tf.Variable([[[1.,2],[3,4]],[[4,5],[6,7]]])

# assume state is 4-d vector [0,0.1,0.2,0.3]

class GRU_onepass(ct.Can):
    def __init__(self,num_h):
        super().__init__()
        # assume input is num_h d.
        Dense = ct.Dense
        self.wz = Dense(num_h+num_h,num_h)
        self.wr = Dense(num_h+num_h,num_h)
        self.w = Dense(num_h+num_h,num_h)
        self.incan([self.wz,self.wr,self.w])
        # http://colah.github.io/posts/2015-08-Understanding-LSTMs/
        
    def __call__(self,i):
        # assume hidden, input is of shape [batch,num_h] and [batch,num_h]
        hidden = i[0]
        inp = i[1]
        wz,wr,w = self.wz,self.wr,self.w
        c = tf.concat([hidden,inp],axis=1)
        z = tf.sigmoid(wz(c))
        r = tf.sigmoid(wr(c))
        h_c = tf.tanh(w(tf.concat([hidden*r,inp],axis=1)))
        h_new = (1-z) * hidden + z * h_c
        return h_new

class GRU(ct.Can):
    def __init__(self,*args):
        super().__init__()
        self.gruo = GRU_onepass(*args)
        def f(last_state, new_input):
            return self.gruo([last_state, new_input])
        self.bscan = ct.BatchScanner(f)
        self.incan([self.gruo,self.bscan])
    def __call__(self,i,*args):
        return self.bscan(i,*args)
        
gruo = GRU_onepass(2)

def f(last_state,new_input):
    new_state = gruo([last_state,new_input])
    return new_state

# def f(last_state,new_input):
#     new_state = tf.concat([last_state[0:2] + new_input, last_state[2:4]],axis=0)
#     return new_state

istate = tf.Variable([0,0.1])
s = ct.BatchScanner(f)
k = s(a,starting_state=None)

k = GRU(2)(a)

sess = ct.get_session()
sess.run(tf.global_variables_initializer())
sess.run(k)

array([[[-0.00191431, -0.00082441],
        [-0.00491235, -0.00271329]],

       [[-0.00498766, -0.0030383 ],
        [-0.00952078, -0.00603461]]], dtype=float32)